In [145]:
from selenium import webdriver
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
from bs4 import BeautifulSoup
import time 
import re
import json 
import pandas as pd
from datetime import datetime

## id 와 password 입력

In [146]:
user_id=""
user_passwd=""

In [147]:
login_option="instagram" # facebook or instagram 
driver_path="chromedriver.exe" 
instagram_id_name="username" 
instagram_pw_name="password"
instagram_login_btn=".sqdOP.L3NKy.y3zKF "
instagram_information_save_name = ".sqdOP.yWX7d.y3zKF"
instagram_alarm_name = ".aOOlW.HoLwm"
is_information_save = False
is_alarm_setting = False

driver = webdriver.Chrome("./chromedriver")

In [148]:
print(f"login start - option {login_option}")
login_url = "https://www.instagram.com/accounts/login/"
driver.get(login_url)
print(login_option)

login start - option instagram
instagram


In [149]:
if login_option == "instagram": 
    time.sleep(2)
    try: 
        instagram_id_form = driver.find_element_by_name(instagram_id_name) 
        instagram_id_form.send_keys(user_id) 
        
        instagram_pw_form = driver.find_element_by_name(instagram_pw_name) 
        instagram_pw_form.send_keys(user_passwd) 
        
        login_ok_button = driver.find_element_by_css_selector(instagram_login_btn) 
        login_ok_button.click() 
        is_login_success = True
        
        time.sleep(3)
        if is_information_save == False and is_alarm_setting == False:
            information_save_btn = driver.find_element_by_css_selector(instagram_information_save_name)
            information_save_btn.click()
            is_information_save = True
            
            alarm_setting_btn = driver.find_element_by_css_selector(instagram_alarm_name)
            alarm_setting_btn.click()
            is_alarm_setting = True
            
    except: 
        print("instagram login fail") 
        is_login_success = False

## 위 내용까지 로그인

In [150]:
my_account_xpath = "/html/body/div[1]/section/nav/div[2]/div/div/div[3]/div/div[6]/span"
my_account_button = driver.find_element_by_xpath(my_account_xpath)
my_account_button.click()

my_profile_full_xpath = "/html/body/div[1]/section/nav/div[2]/div/div/div[3]/div/div[6]/div[2]/div[2]/div[2]/a[1]/div"
my_profile_button = driver.find_element_by_xpath(my_profile_full_xpath)
my_profile_button.click()

In [151]:
time.sleep(1.5)
# 게시물 개수
post_count = driver.find_element_by_xpath("/html/body/div[1]/section/main/div/header/section/ul/li[1]/div/span").text
post_count = int(post_count)

print("게시물 수 : ",post_count)

게시물 수 :  13


### 현재 모든 댓글보기 클릭 가능 (while문 이용)
<li> 1. 모든 답글이 보여진 상태에서 html parser 이용
<li> 2. css 또는 xpath 이용

In [154]:
# 첫 게시물
first_post_css = "a > div.eLAPa > div._9AhH0"
first_post_button = driver.find_element_by_css_selector(first_post_css)
first_post_button.click()

for i in range(post_count) :
    # 날짜 데이터 크롤링
    time.sleep(1.5)
    #date = driver.find_element_by_css_selector("div.k_Q0X.I0_K8.NnvRN > a > time").text
    date = driver.find_element_by_css_selector('time.FH9sR.Nzb55')
    date_info = pd.to_datetime(date.get_attribute('datetime')).normalize()
    date = str(date_info.year) + "년 " + str(date_info.month) + "월 " + str(date_info.day) +"일"
    print("date : ", date)

    # 장소 크롤링
    time.sleep(0.5)
    try : 
        place = driver.find_element_by_css_selector("body > div.RnEpo._Yhr4 > div.pbNvD.QZZGH.bW6vo > div > article > div > div.HP0qD > div > div > div.UE9AK > div > header > div.o-MQd.z8cbW > div.M30cS > div > div.JF9hh > div > a").text
    except : 
        place = None
    print("place : ",place)
    
    # 게시글 내용 크롤링
    #body > div.RnEpo._Yhr4 > div.pbNvD.QZZGH.bW6vo > div > article > div > div.HP0qD > div > div > div.eo2As > div.EtaWk > ul
    try : 
        post_content = driver.find_element_by_xpath("/html/body/div[6]/div[3]/div/article/div/div[2]/div/div/div[2]/div[1]/ul/div/li/div/div/div[2]/div[1]/span").text    
    except :
        post_content = None
    # 정제 필요
    print("original post_content : ", post_content)
    
    # 본문 속 해시태그 정제
    if post_content != None :
        post_content_tag= []
        post_content = re.findall("#[A-Za-z0-9가-힣]+", post_content)
        content = ''.join(post_content).replace("#"," ") # "# 제거"
        content_split = content.split()
        
        for tag in content_split :
            post_content_tag.append(tag)
    print("tag in post_content : ", post_content_tag)
    
    # 댓글 크롤링
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    comment_count = 0

    # 댓글 있는지 확인
    try  :
        ul_list = soup.find_all('ul',  'Mr508')
        comment_count = len(ul_list)
    except :
        # 댓글 없으면
        comment_count = 0
    else : 
        # 댓글 개수 적용하여 대댓글 열기버튼 xpath 추출
        for i in range(1,comment_count+1) :
            button_xpath = "/html/body/div[6]/div[3]/div/article/div/div[2]/div/div/div[2]/div[1]/ul/ul[" + str(i) + "]/li/ul/li/div/button"
            try : 
                # 대댓글 열기버튼 있으면 클릭
                driver.find_element_by_xpath(button_xpath).click()
                time.sleep(1)
                button_check = driver.find_element_by_class_name("EizgU").text
                while "답글 숨기기" not in button_check : 
                    driver.find_element_by_xpath(button_xpath).click()
                    button_check = driver.find_element_by_class_name("EizgU").text
                    
            except :
                button_xpath = None
            
        # 대댓글이 모두 열려있는 상태에서 html parser        
        # 추가 댓글이 모두 열려있는 상태에서 html parser        
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        ul_list = soup.find_all("ul","Mr508")
        for d in ul_list :
            user_id = d.find("h3","_6lAjh").find("a").get_text()
            print("user id : ",user_id)
            comment = d.find("div","C4VMK").find("div","MOdxS").get_text()
            print("comment : " , comment)
            try :
                re_ul_list = d.find_all("ul","TCSYW")
                for dd in re_ul_list :
                    print("[ 답글시작 ]")
                    try : 
                        re_div_list = dd.find_all("div","ZyFrc")
                        for ddd in re_div_list :
                            print("[답글의 답글 시작]")
                            comment_user_id = ddd.find("h3","_6lAjh").find("a").get_text()
                            print("comment_user_id : ",comment_user_id)
                            comment_comment = ddd.find("div","C4VMK").find("div","MOdxS").get_text()
                            print("comment_comment : " , comment_comment)
                        print("\n")
                    except : 
                        re_div_list =""
            except : 
                re_ul_list = ""
            
    # 다음 버튼
    try :
        next_post_button = driver.find_element_by_css_selector("div.RnEpo._Yhr4 > div.Z2Inc._7c9RR > div > div.l8mY4.feth3 > button")
        next_post_button.click()
        time.sleep(0.5)
    except :
        # 닫기 버튼    
        exit_button = driver.find_element_by_css_selector("div.RnEpo._Yhr4 > div.NOTWr > button > div > svg")
        exit_button.click()
        break

date :  2022년 2월 15일
place :  None
original post_content :  #문자메세지
tag in post_content :  ['문자메세지']
user id :  ___kiss_o
comment :  이거 테스트 ㄱ
user id :  ___kiss_o
comment :  #나는지금기쁘다
[ 답글시작 ]
[답글의 답글 시작]
comment_user_id :  go_ne_jin
comment_comment :  @___kiss_o #나는지금아무생각이없다
[답글의 답글 시작]
comment_user_id :  go_ne_jin
comment_comment :  .
[답글의 답글 시작]
comment_user_id :  go_ne_jin
comment_comment :  .
[답글의 답글 시작]
comment_user_id :  go_ne_jin
comment_comment :  .
[답글의 답글 시작]
comment_user_id :  go_ne_jin
comment_comment :  .
[답글의 답글 시작]
comment_user_id :  go_ne_jin
comment_comment :  .
[답글의 답글 시작]
comment_user_id :  go_ne_jin
comment_comment :  .
[답글의 답글 시작]
comment_user_id :  ___kiss_o
comment_comment :  @go_ne_jin 학생 지금 뭐하는거죠?
[답글의 답글 시작]
comment_user_id :  go_ne_jin
comment_comment :  @___kiss_o 댓글크롤링 모든댓글 수집하려면 버튼 계속눌러야되더라구요????🤦🏻‍♀️


user id :  jo_o_han
comment :  ?ㅡ?
[ 답글시작 ]
[답글의 답글 시작]
comment_user_id :  go_ne_jin
comment_comment :  @jo_o_han 뿌링클
[답글의 답글 시작]
comment_user_id :  go_ne_j

date :  2020년 8월 12일
place :  해운대 - Haeundae
original post_content :  바다는 힐링🌙
.
.
.
.
.
.
.
근데 비와서 추어
tag in post_content :  []
user id :  __j.euniii__
comment :  여행갔는데 홀쭉해졌는데?! 좋겠따 여러모로😏😏
user id :  xiyehn
comment :  예뻐
[ 답글시작 ]
[답글의 답글 시작]
comment_user_id :  go_ne_jin
comment_comment :  @xiyehn 아 칭찬들어어써 종강하면 바로 파주다


user id :  minzzee_
comment :  아 진짜 개부럽다아아아아ㅏㄱ
user id :  myez__
comment :  와 부러워요
[ 답글시작 ]
[답글의 답글 시작]
comment_user_id :  go_ne_jin
comment_comment :  @myez__ 성의없자나.


user id :  y.hy_712
comment :  우와 인어공주다ㅏ
[ 답글시작 ]
[답글의 답글 시작]
comment_user_id :  go_ne_jin
comment_comment :  @yang_is_sheep 하지말라고 부계정으로 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ
[답글의 답글 시작]
comment_user_id :  y.hy_712
comment_comment :  @go_ne_jin ㅋㅅㅋㄹㅋㅋㅋㅋㅋ 이게 유행이라고


user id :  go_ne_jin
comment :  #해운대 #해수 #욕 #장 #20.08.12
date :  2020년 7월 24일
place :  샤로수길
original post_content :  나도 모라도 올릴래
tag in post_content :  []
date :  2020년 5월 28일
place :  월미도
original post_content :  바다조아~ 아 좋아좋아🌊
tag in post_content :  []
user id 

## 아래는 첫번째 게시글로만 test하는 코드

In [116]:
# 첫 게시물
first_post_css = "a > div.eLAPa > div._9AhH0"
first_post_button = driver.find_element_by_css_selector(first_post_css)
first_post_button.click()

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
comment_count = 0

# 댓글 있는지 확인
try  :
    ul_list = soup.find_all('ul',  'Mr508')
    comment_count = len(ul_list)
except :
    # 댓글 없으면
    comment_count = 0
else : 
    # 댓글 개수 적용하여 대댓글 열기버튼 xpath 추출
    for i in range(1,comment_count+1) :
        button_xpath = "/html/body/div[6]/div[3]/div/article/div/div[2]/div/div/div[2]/div[1]/ul/ul[" + str(i) + "]/li/ul/li/div/button"
        try : 
            # 대댓글 열기버튼 있으면 클릭
            driver.find_element_by_xpath(button_xpath).click()
            button_check = driver.find_element_by_class_name("EizgU").text
            time.sleep(1)
            while "답글 숨기기" not in button_check : 
                driver.find_element_by_xpath(button_xpath).click()
                button_check = driver.find_element_by_class_name("EizgU").text
                    
        except :
            button_xpath = None

    # 추가 댓글이 모두 열려있는 상태에서 html parser        
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    ul_list = soup.find_all("ul","Mr508")
    for d in ul_list :
        user_id = d.find("h3","_6lAjh").find("a").get_text()
        print("user id : ",user_id)
        comment = d.find("div","C4VMK").find("div","MOdxS").get_text()
        print("comment : " , comment)
        try :
            re_ul_list = d.find_all("ul","TCSYW")
            for dd in re_ul_list :
                print("[ 답글시작 ]")
                print(dd)
                print("\n")
                try : 
                    re_div_list = dd.find_all("div","ZyFrc")
                    for ddd in re_div_list :
                        print("[답글의 답글 시작]")
                        comment_user_id = ddd.find("h3","_6lAjh").find("a").get_text()
                        print("comment_user_id : ",comment_user_id)
                        comment_comment = ddd.find("div","C4VMK").find("div","MOdxS").get_text()
                        print("comment_comment : " , comment_comment)
                    print("\n")
                except : 
                    re_div_list =""
        except : 
            re_ul_list = ""

user id :  ___kiss_o
comment :  이거 테스트 ㄱ
user id :  ___kiss_o
comment :  #나는지금기쁘다
user id :  jo_o_han
comment :  ?ㅡ?
user id :  perarduaadastra__
comment :  이게뭐에요?
user id :  ___kiss_o
comment :  이거 테스트 ㄱ
user id :  ___kiss_o
comment :  #나는지금기쁘다
[ 답글시작 ]
<ul class="TCSYW"><li class="_61Di1"><div class="qF0y9 Igw0E IwRSH eGOV_ ybXk5 _4EzTm MGdpg"><button class="sqdOP yWX7d y3zKF" type="button"><div class="_7mCbS"></div><span class="EizgU">답글 숨기기</span></button></div></li><div class="ZyFrc" role="button" tabindex="0"><li class="gElp9 rUo9f lDe-V" role="menuitem"><div class="P9YgZ"><div class="C7I1f"><div class="Jv7Aj mArmR pZp3x"><div aria-disabled="false" class="RR-M- h5uC0 TKzGu" role="button" style="cursor: pointer;" tabindex="0"><canvas class="CfWVH" height="50" style="position: absolute; top: -4px; left: -4px; width: 40px; height: 40px;" width="50"></canvas><span class="_2dbep" role="link" style="width: 32px; height: 32px;" tabindex="-1"><img alt="go_ne_jin님의 프로필 사진" class="_6q-tv" 

## html parser test

In [104]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
ul_list = soup.find_all("ul","Mr508")
for d in ul_list :
    user_id = d.find("h3","_6lAjh").find("a").get_text()
    print("user id : ",user_id)
    comment = d.find("div","C4VMK").find("div","MOdxS").get_text()
    print("comment : " , comment)
    try :
        re_ul_list = d.find_all("ul","TCSYW")
        for dd in re_ul_list :
            print("[ 답글시작 ]")
            #print(dd)
            try : 
                re_div_list = dd.find_all("div","ZyFrc")
                for ddd in re_div_list :
                    print("[답글의 답글 시작]")
                    comment_user_id = ddd.find("h3","_6lAjh").find("a").get_text()
                    print("comment_user_id : ",comment_user_id)
                    comment_comment = ddd.find("div","C4VMK").find("div","MOdxS").get_text()
                    print("comment_comment : " , comment_comment)
                print("\n")
            except : 
                re_div_list =""
    except : 
        re_ul_list = ""



user id :  ___kiss_o
comment :  이거 테스트 ㄱ


user id :  ___kiss_o
comment :  #나는지금기쁘다
[ 답글시작 ]
[답글의 답글 시작]
comment_user_id :  go_ne_jin
comment_comment :  @___kiss_o #나는지금아무생각이없다
[답글의 답글 시작]
comment_user_id :  go_ne_jin
comment_comment :  .
[답글의 답글 시작]
comment_user_id :  go_ne_jin
comment_comment :  .
[답글의 답글 시작]
comment_user_id :  go_ne_jin
comment_comment :  .
[답글의 답글 시작]
comment_user_id :  go_ne_jin
comment_comment :  .
[답글의 답글 시작]
comment_user_id :  go_ne_jin
comment_comment :  .
[답글의 답글 시작]
comment_user_id :  go_ne_jin
comment_comment :  .
[답글의 답글 시작]
comment_user_id :  ___kiss_o
comment_comment :  @go_ne_jin 학생 지금 뭐하는거죠?
[답글의 답글 시작]
comment_user_id :  go_ne_jin
comment_comment :  @___kiss_o 댓글크롤링 모든댓글 수집하려면 버튼 계속눌러야되더라구요????🤦🏻‍♀️




user id :  jo_o_han
comment :  ?ㅡ?
[ 답글시작 ]
[답글의 답글 시작]
comment_user_id :  go_ne_jin
comment_comment :  @jo_o_han 뿌링클
[답글의 답글 시작]
comment_user_id :  go_ne_jin
comment_comment :  @jo_o_han 시급




user id :  perarduaadastra__
comment :  이게뭐에요?
[ 답글시작 ]